# Testing Postgres database

In [ ]:
import psycopg2
import pandas as pd

In [ ]:
# Database connection parameters
DB_HOST = "217.71.200.156"
DB_PORT = "5433"
DB_NAME = "esesa"
DB_USER = "esesa"
DB_PASSWORD = "esesa"


In [ ]:
# Establish connection
def connect_to_db():
    try:
        conn = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Connection successful")
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

In [ ]:
# Example query execution
def execute_query(query):
    conn = connect_to_db()
    if conn:
        try:
            df = pd.read_sql_query(query, conn)
            conn.close()
            return df
        except Exception as e:
            print(f"Query execution error: {e}")
            conn.close()
            return None
    return None

In [ ]:
# Example queries

# Customer with the most money spent"
query_1 = """
with customer_total_rentals as (
                SELECT 
                    customer_id,
                    count(rental_id) as total_rentals,
                    sum(amount) as total_amount
                FROM fact_rental_payment
                GROUP BY customer_id
                )

                SELECT 
                    customer_id,
                    total_rentals,
                    total_amount
                FROM customer_total_rentals
                WHERE total_amount = (SELECT max(total_amount) FROM customer_total_rentals);
"""

# City where the store with the most sales is located"
query_2 = """
    with store_total_sales as (
                    SELECT store_id
                            ,sum(amount) as total_sales
                    FROM fact_rental_payment
                    GROUP BY store_id
                    ),

                    store_city as (
                        SELECT 
                            s.store_id,
                            s.address_id,
                            a.city_id,
                            c.city
                        FROM store s,
                        address a,
                        city c 
                        WHERE s.address_id = a.address_id
                        AND a.city_id = c.city_id
                    )

                    SELECT 
                        st.city,
                        s.store_id,
                        s.total_sales
                    FROM store_total_sales s 
                    JOIN store_city st ON s.store_id = st.store_id
                    WHERE total_sales = (SELECT max(total_sales) FROM store_total_sales);
"""
    
result_1 = execute_query(query_1)
result_2 = execute_query(query_2)

if result_1 is not None:
    print("Query 1 result:")
    print(result_1)

if result_2 is not None:
    print("Query 2 result:")
    print(result_2)


Connection successful


/tmp/ipykernel_39084/814106468.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Connection successful
Query 1 result:
   customer_id  total_rentals  total_amount
0          526             45        221.55
Query 2 result:
        city  store_id  total_sales
0  Woodridge         2     33726.77
